## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py37h89c1867_0        1010 KB  conda-forge
    toolz-0.12.1               |     pyhd8ed1ab_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  toolz              conda-forge/noarch::toolz-0.12.1-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.12.0-py37h89c1867_0 --> 4.14.0-py37h89c1867_0



conda-4.14.0         | 1010 KB   | ##################################### | 100% 
toolz-0.12.1         | 51 KB     | ##################################### | 100% 
Preparing transaction: done
Ver

## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(5, 6):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:34<00:00,  4.66it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_5"])
layer_hs_array.shape

(10000, 267264)

## Layer 5 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

dim_reduced_vecs = layer_hs_array

# random_projector = GaussianRandomProjection(random_state = 42)
# dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 267264)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 267264D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 1.65 s
Outer iteration 0 / 10
  Iteration 19 (27.43 s, search 18.68 s): objective=5399.29 imbalance=1.664 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (54.80 s, search 37.41 s): objective=5374.6 imbalance=2.084 nsplit=0        
Outer iteration 2 / 10
  Iteration 19 (82.39 s, search 56.12 s): objective=5404.21 imbalance=1.386 nsplit=0       
Objective improved: keep new clusters
Outer iteration 3 / 10
  Iteration 19 (109.79 s, search 74.78 s): objective=5392.6 imbalance=1.473 nsplit=0        
Outer iteration 4 / 10
  Iteration 19 (137.50 s, search 93.54 s): objective=5399.66 imbalance=1.208 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (164.84 s, search 112.25 s): objective=5388.75 imbalance=1.970 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (192.24 s, search 130.98 s): objective=5378.77 imbalance=1.700 nsplit=0       
Outer i

5416.0771484375

In [10]:
kmeans.centroids #cluster centers

array([[-1.78076401e-02,  2.24871077e-02,  9.42376908e-03, ...,
         2.03777859e-06,  5.02010425e-06, -4.69501038e-06],
       [-2.16561258e-02,  2.73468532e-02,  1.14603452e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.80298742e-02,  2.27677412e-02,  9.54137836e-03, ...,
        -1.94792356e-05,  1.09488265e-05, -1.06546153e-07],
       ...,
       [-1.93852279e-02,  2.44792942e-02,  1.02586402e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.85471121e-02,  2.34209225e-02,  9.81509965e-03, ...,
         1.79142967e-06,  4.27059831e-06,  7.51854441e-06],
       [-2.10353602e-02,  2.65629925e-02,  1.11318575e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

1.0000219
1.0000107
1.0000224
1.000012
1.0000097
1.0000207
1.0000223
1.0000184
1.0000298
1.0000105


In [12]:
kmeans.obj #inertia at each iteration

array([3429.27539062, 5219.87646484, 5332.78710938, 5366.20361328,
       5379.78515625, 5390.65087891, 5394.82714844, 5396.09375   ,
       5396.59765625, 5396.90087891, 5397.08251953, 5397.26416016,
       5397.4765625 , 5397.82373047, 5398.48046875, 5399.08447266,
       5399.25585938, 5399.26708984, 5399.28808594, 5399.29296875,
       3567.99487305, 5203.7265625 , 5273.83398438, 5339.9296875 ,
       5350.80664062, 5354.62548828, 5357.17285156, 5358.46289062,
       5359.23632812, 5360.12744141, 5362.37890625, 5368.21337891,
       5373.265625  , 5374.22119141, 5374.43798828, 5374.51318359,
       5374.56396484, 5374.58544922, 5374.60888672, 5374.60302734,
       3350.9074707 , 5218.41650391, 5325.09472656, 5358.69873047,
       5371.93847656, 5381.55224609, 5386.67480469, 5389.39941406,
       5391.35791016, 5392.84179688, 5394.26074219, 5395.74511719,
       5398.01757812, 5400.640625  , 5401.56201172, 5402.95117188,
       5404.04003906, 5404.12207031, 5404.16503906, 5404.20947

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

1    2102
7    1677
6    1455
4    1067
3    1036
0     755
2     545
5     517
8     430
9     416
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

1    2102
7    1677
6    1455
4    1067
3    1036
0     755
2     545
5     517
8     430
9     416
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [18]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[1, 6, 3, 1, 4, 1, 1, 4, 1, 2]

In [19]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

[5, 4, 10, 3, 5, 9, 6, 6, 3, 10]

In [20]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

max_labels = [label for label in max_label_per_centroid for _ in range(1000)]

In [21]:
np.array(max_centroids)

array([1, 1, 1, ..., 2, 2, 2])

In [22]:
np.array(max_labels)

array([ 5,  5,  5, ..., 10, 10, 10])

In [23]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [24]:
label_to_centroid

{1: 1, 2: 6, 3: 3, 4: 1, 5: 4, 6: 1, 7: 1, 8: 4, 9: 1, 10: 2}

In [25]:
centroid_to_label

{0: 5, 1: 4, 2: 10, 3: 3, 4: 5, 5: 9, 6: 6, 7: 6, 8: 3, 9: 10}

In [26]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

array([ 6,  9,  6, ..., 10, 10, 10])

In [27]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.0
Prompt 2 Accuracy:  0.0
Prompt 3 Accuracy:  0.965
Prompt 4 Accuracy:  0.403
Prompt 5 Accuracy:  0.764
Prompt 6 Accuracy:  0.511
Prompt 7 Accuracy:  0.0
Prompt 8 Accuracy:  0.0
Prompt 9 Accuracy:  0.182
Prompt 10 Accuracy:  0.955


In [28]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))

Overall Accuracy:  0.378


In [29]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications[i * 1000:(i + 1) * 1000] == max_centroids[i * 1000:(i + 1)*1000]))

Prompt 1 Accuracy:  0.211
Prompt 2 Accuracy:  0.249
Prompt 3 Accuracy:  0.546
Prompt 4 Accuracy:  0.403
Prompt 5 Accuracy:  0.424
Prompt 6 Accuracy:  0.402
Prompt 7 Accuracy:  0.402
Prompt 8 Accuracy:  0.212
Prompt 9 Accuracy:  0.31
Prompt 10 Accuracy:  0.545


In [30]:
print(f"Overall Accuracy: ", np.mean(classifications == max_centroids))

Overall Accuracy:  0.3704


## Measure Conversion Accuracy

In [31]:
label_to_centroid_vectorized_map = np.vectorize(label_to_centroid.get)
centroid_to_label_vectorized_map = np.vectorize(centroid_to_label.get)
prompt_ids_to_centroids = label_to_centroid_vectorized_map(prompt_ids)
max_centroids_to_labels = centroid_to_label_vectorized_map(max_centroids)

In [32]:
np.mean(max_centroids == prompt_ids_to_centroids) #100% - max_centroids is the prompt ids converted to the max centroid corresponding to each label

1.0

In [33]:
np.mean(max_centroids_to_labels == prompt_ids) #<100%

0.4